In [ ]:
from pm4py import *
from pm4py.algo.simulation.playout.petri_net import algorithm as simulator
from pm4py.visualization.petri_net import visualizer as pn_visualizer
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.evaluation.replay_fitness import algorithm as replay_fitness
from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.visualization.process_tree import visualizer as pt_visualizer
from pm4py.visualization.heuristics_net import visualizer as hn_visualizer
from pm4py.visualization.dfg import visualizer as dfg_visualization
from pm4py.objects.conversion.process_tree import converter as pt_converter

import pandas as pd
from datetime import date
import seaborn as sns

## 1.Загрузка бизнес-процесса и преобразование его в сеть Петри

In [ ]:
bpmn_graph = read_bpmn('uni.bpmn')
net,im,fm = convert_to_petri_net(bpmn_graph)

gviz = pn_visualizer.apply(
    net,
    im,
    fm,
    parameters={"debug": True, "set_rankdir": "LR"} # set_rankdir for horizontal layout
)

# Render and view the graph
pn_visualizer.view(gviz)


In [ ]:
#from pm4py.algo.analysis.woflan.graphs import minimal_coverability_graph
#import os


#mcg=minimal_coverability_graph.minimal_coverability_graph.minimal_coverability_tree(net, im, fm)

#from pm4py.algo.analysis.woflan.graphs.minimal_coverability_graph import apply as minimal_coverability_graph
#mcg = minimal_coverability_graph(net, initial_marking)




## 2.Построение графа покрытия и достижимости маркировок


In [ ]:

net, initial_marking, final_marking = convert_to_petri_net(bpmn_graph)

from pm4py.objects.petri_net.utils import reachability_graph

#конструируем граф достижимости встроенными средствами 
ts=reachability_graph.construct_reachability_graph(net,initial_marking)

from pm4py.visualization.transition_system import visualizer as ts_visualizer

gviz=ts_visualizer.apply(ts,parameters={ts_visualizer.Variants.VIEW_BASED.value.Parameters.FORMAT:"png"})
ts_visualizer.view(gviz)



In [ ]:
#построения графа покрытия маркировок и графа достижимости маркировок срдествами woflan 

from pm4py.algo.analysis.woflan.graphs.minimal_coverability_graph import minimal_coverability_graph
from pm4py.algo.analysis.woflan.graphs.reachability_graph import reachability_graph
import pygraphviz as pgv 
from IPython.display import Image, display
from matplotlib import pyplot as plt

#граф покрытия 
cg=minimal_coverability_graph.apply(net, initial_marking, final_marking)

#граф достижимости
rg=reachability_graph.apply(net, initial_marking, final_marking)


    



In [ ]:
def vizualizeGraph(network):#визуализация графа покрытия маркировок и проверка на неограниченность
    is_bounded=True
    GR = pgv.AGraph(strict=False, directed=True)
    GR.graph_attr['rankdir'] = 'LR'
    GR.node_attr['shape'] = 'ellipse'


    for node, attributes in network.nodes(data=True):
        marking = attributes['marking']
        for pos in marking:
            if(pos==float('inf')):
                is_bounded=False
              
        GR.add_node(node, label=marking)
        
    for e in network.edges(data=True):
        GR.add_edge(e[0], e[1], label=e[2]['transition'], penwidth=1)
 

    GR.layout(prog='dot')  # Layout algorithm (e.g., 'dot', 'neato', 'fdp', 'circo')
    display(Image(GR.draw(format='png')))
    return is_bounded

vizualizeGraph(cg)

In [ ]:
vizualizeGraph(rg)

In [ ]:
from pm4py.objects.petri_net.utils import petri_utils
from pm4py.objects.petri_net.exporter import exporter as pnml_exporter
from pm4py.visualization.petri_net import visualizer as pn_visualizer

from pm4py.objects.petri_net.obj import PetriNet, Marking



import copy


# делаем копию сети
net_copy = copy.deepcopy(net)



# добавляем X - переход и проверяем на ограниченность 

def add_cycle_transition(pn: PetriNet, first_place: PetriNet.Place, last_place: PetriNet.Place):
    """
    Добавляет переход, который берёт токен из последнего места
    и кладёт его в первое место.
    """
    # создаём переход
    t_cycle = PetriNet.Transition("t_cycle", "t_cycle")

    # добавляем переход в сеть
    pn.transitions.add(t_cycle)

    # дуги: last_place → t_cycle → first_place
    petri_utils.add_arc_from_to(last_place, t_cycle, pn)
    petri_utils.add_arc_from_to(t_cycle, first_place, pn)
    return t_cycle


#вычисляем начальные и конечные позиции
begin_places = [place for place, tokens in im.items() if tokens > 0]

final_places = [place for place, tokens in fm.items() if tokens > 0]

# создаём переход
t_cycle = PetriNet.Transition("t_cycle", "t_cycle")

# добавляем переход в сеть
net_copy.transitions.add(t_cycle)


#получить реальную позицию (объект) по ключу из маркировки

def get_place_by_name(pn, name):
    for place in pn.places:
        if place.name == name:
            return place
    return None  # если места с таким именем нет




p_fin = next(iter(fm.keys()))   # объект Place
p_beg = next(iter(im.keys()))   # объект Place
p_Fin = get_place_by_name (net_copy, p_fin.name)
p_Beg = get_place_by_name (net_copy, p_beg.name)


#добавляем дуги
petri_utils.add_arc_from_to(p_Fin, t_cycle, net_copy)
petri_utils.add_arc_from_to( t_cycle,p_Beg, net_copy)


#покажем расширенную сетку
gviz = pn_visualizer.apply(
    net_copy,
    im,
    fm,
    parameters={"debug": True, "set_rankdir": "LR"} # set_rankdir for horizontal layout
)

# отобразим 
pn_visualizer.view(gviz)


# строим граф покрытия маркировок расширенной сети  и там же проверяем на ограниченность (нет омег)

cg_ext=minimal_coverability_graph.apply(net_copy, initial_marking, final_marking)
is_bounded=vizualizeGraph(cg_ext)
print("Is bounded?:",is_bounded)

In [ ]:
import networkx as nx

#TODO: надо добавить проверку на ограниченность. Вопрос - зачем?

rg_ext=reachability_graph.apply(net_copy, initial_marking, final_marking)
vizualizeGraph(rg_ext)

#проверяем на живость расширенную сеть
#вычисляем сильносвязные компоненты. Для живости компонента должна быть одна и каждый переход сети должен ей принадлежать 


scc = list(nx.strongly_connected_components(rg_ext))
if len(scc)==1:
    print("Is live")
else:
    print("Not live")

# Вопрос - ДОПОЛНИТЬ проверкой, что каждый переход сетки входит в компоненту SCC (Задание) 
print(scc)

# Если компонента сильной связности расширенной сети Петри одна и ей принадлежат все переходы, то делаем вывод, что сеть живая  
#
#



## 3.Анализ бездефектности 
**WOFLAN**

In [ ]:
net, initial_marking, final_marking = convert_to_petri_net(bpmn_graph)

from pm4py.algo.analysis.woflan import algorithm as woflan

is_sound = woflan.apply(net, initial_marking, final_marking, parameters={woflan.Parameters.RETURN_ASAP_WHEN_NOT_SOUND: True,
                                                     woflan.Parameters.PRINT_DIAGNOSTICS: True,
                                                                          
                                                     woflan.Parameters.RETURN_DIAGNOSTICS: True})

print("\nIs Workflow net sound? ->", is_sound[0])

print("\n Полная статистика",is_sound[1])


## 4.Генерация журнала событий 

In [ ]:
simulated_log = simulator.apply(net, initial_marking, variant=simulator.Variants.BASIC_PLAYOUT,
                                    parameters={simulator.Variants.BASIC_PLAYOUT.value.Parameters.NO_TRACES: 50})

write_xes(simulated_log, 'log.xes')

gviz = pn_visualizer.apply(net, initial_marking, final_marking)
pn_visualizer.view(gviz)

dataframe = convert_to_dataframe(simulated_log)
dataframe.to_csv('exp.csv')


In [ ]:
df = pd.read_csv("exp.csv", index_col=0)
df = df.rename(columns={"case:concept:name": "client", "time:timestamp": "datetime", "concept:name": "action"})
df['resource']=''
df.head(20)

In [ ]:
student_case = ['Выбрать направление', 
                  'Проходить курсы', 
                  'Выбрать проекты',
                  'Выбрать доп. услуги',
                  'Сделать проекты',
                  'Ходить на языковую подготовку',
                  'Ходить в спортзал']

council_case = ['Регистрация заявления студента', 
                'Рассмотреть заявление',
                'Принять решение по заявлению',
                'Проверить ранее пройденные дисциплины',
                'Отправить уведомления о зачислении']

prof_case = ['Решить насчет дат',
                'Назначить другой день сдачи',
                'Согласовать проекты',
                'Проверить работы',
                'Поставить оценку']

for row_inx in range(df.shape[0]):
    if df.iloc[row_inx, 0] in student_case:
        df.loc[row_inx, 'resource'] = 'Студент'
    elif df.iloc[row_inx, 0] in council_case:
        df.loc[row_inx, 'resource'] = 'Приемная комиссия'
    elif df.iloc[row_inx, 0] in prof_case:
        df.loc[row_inx, 'resource'] = 'Преподаватель'

In [ ]:
df.head(20)

In [ ]:
df.to_csv('log.csv')

## 5.Предварительный анализ журнала событий

In [ ]:
events = pd.read_csv('log.csv', index_col=0)
events['datetime'] = pd.to_datetime(events['datetime'])
events.head()

In [ ]:
print(f'Event log has {events.shape[0]} rows and {events.shape[1]} columns.')

In [ ]:
## Get the case start times to get the time deltas for the 'age' of each activity with respect to start
case_starts_ends = events.pivot_table(index='client', aggfunc={'datetime': ['min', 'max']})
case_starts_ends = case_starts_ends.reset_index()
case_starts_ends.columns = ['client', 'caseend', 'casestart']
events = events.merge(case_starts_ends, on='client')
events['relativetime'] = (events['caseend']) - events['casestart']

## Get day of week
events['weekday'] = events['datetime'].apply(lambda x: x.weekday())
events['date'] = events['datetime'].apply(lambda x: x.date())
events['startdate'] = events['casestart'].apply(lambda x: x.date())
events['hour'] = events['datetime'].apply(lambda x: x.time().hour)
## Get relative times in more friendly terms
events['relativetime_s'] = events['relativetime'].dt.seconds + 86400*events['relativetime'].dt.days
events['relativedays'] = events['relativetime'].dt.days

events.head()

### Точечная диаграмма

In [ ]:
patientnums = [int(e) for e in events['client'].apply(lambda x: str(x).strip('client'))]
resourcenums = [i for (i, e) in enumerate(events['resource'])]

In [ ]:
ax = sns.scatterplot(x=events['relativetime_s'], y=events['client'], hue=events['action'])
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)

In [ ]:
# test_df = events[events['action'] == 'Принятие заявки']
# patientnums = [int(e) for e in test_df['client'].apply(lambda x: str(x).strip('client'))]
# resourcenums = [i for (i, e) in enumerate(test_df['resource'])]
# ax = sns.scatterplot(x=test_df['relativetime_s'], y=test_df['client'], hue=test_df['action'])
# ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)

In [ ]:
ax = sns.stripplot(x=events['weekday'], y=resourcenums, hue=events['action'], jitter=0.2)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1)

## 6.Реконструкция процесса алгоритмами ProcessMining

### Альфа майнер

In [ ]:
# alpha miner
simulated_log = read_xes('log.xes')

net, initial_marking, final_marking = alpha_miner.apply(simulated_log)

# Visualise
parameters = {pn_visualizer.Variants.FREQUENCY.value.Parameters.FORMAT: "png"}
gviz = pn_visualizer.apply(net, initial_marking, final_marking,
                            parameters=parameters,
                            variant=pn_visualizer.Variants.FREQUENCY,
                            log=simulated_log)
pn_visualizer.view(gviz)

### Альфа+ майнер

In [ ]:
# alpha miner
simulated_log = read_xes('log.xes')

net, initial_marking, final_marking = discover_petri_net_alpha_plus(simulated_log)

# Visualise
parameters = {pn_visualizer.Variants.FREQUENCY.value.Parameters.FORMAT: "png"}
gviz = pn_visualizer.apply(net, initial_marking, final_marking,
                            parameters=parameters,
                            variant=pn_visualizer.Variants.FREQUENCY,
                            log=simulated_log)
pn_visualizer.view(gviz)

### Эвристический майнер

In [ ]:
# heuristics miner
simulated_log = read_xes('log.xes')

heu_net = heuristics_miner.apply_heu(simulated_log)

# Visualise
gviz = hn_visualizer.apply(heu_net)
hn_visualizer.view(gviz)

#Petri-net of heuristic miner output
# heuristics miner
net, im, fm = heuristics_miner.apply(simulated_log)

# viz
gviz = pn_visualizer.apply(net, im, fm)

# pn_visualizer.view(gviz)
pn_visualizer.view(gviz)

### Индуктивный майнер

In [ ]:
# Загрузка XES лог-файла
simulated_log = xes_importer.apply('log.xes')

# Построение дерева процессов
tree = inductive_miner.apply(simulated_log)

# Визуализация дерева процессов
gviz = pt_visualizer.apply(tree)
pt_visualizer.view(gviz)

## 7.Вычисление метрик качества (проверка соответствия)

In [ ]:
from pm4py.algo.conformance.tokenreplay import algorithm as conformance_diagnostics_token_based_replay

# Загрузка XES лог-файла
simulated_log = xes_importer.apply('log.xes')

# Построение дерева процессов с помощью индуктивного майнера
tree = inductive_miner.apply(simulated_log)

# Преобразуем дерево процессов в сеть Петри
net3, im3, fm3 = pt_converter.apply(tree)

# Получаем сеть Петри альфа-алгоритмом
net, im, fm = alpha_miner.apply(simulated_log)

# Получаем сеть Петри эвристическим алгоритмом
net2, im2, fm2 = heuristics_miner.apply(simulated_log)

# Проверяем качество реконструкции альфа-алгоритмом
replayed_traces1 = conformance_diagnostics_token_based_replay.apply(simulated_log, net, im, fm)

# Проверяем качество реконструкции эвристическим алгоритмом
replayed_traces2 = conformance_diagnostics_token_based_replay.apply(simulated_log, net2, im2, fm2)

# Проверяем качество реконструкции индуктивным алгоритмом (через преобразованную сеть Петри)
replayed_traces3 = conformance_diagnostics_token_based_replay.apply(simulated_log, net3, im3, fm3)

In [ ]:
print(replayed_traces1[0]["trace_is_fit"])
print(replayed_traces2[0]["trace_is_fit"])
print(replayed_traces3[0]["trace_is_fit"])

In [ ]:
log_fitness1 = replay_fitness.evaluate(replayed_traces1, variant=replay_fitness.Variants.TOKEN_BASED)
log_fitness2 = replay_fitness.evaluate(replayed_traces2, variant=replay_fitness.Variants.TOKEN_BASED)
log_fitness3 = replay_fitness.evaluate(replayed_traces3, variant=replay_fitness.Variants.TOKEN_BASED)

In [ ]:
log_fitness1, log_fitness2, log_fitness3



### Выводы из результатов:
**Альфа-алгоритм (log_fitness1):**
   - Низкий perc_fit_traces (0%) означает, что построенная альфа-алгоритмом модель не соответствует журналу событий.
   - Средний уровень соответствия (average_trace_fitness = 0.8284088969002013) высок, что говорит о том, что трассы, которые модель смогла воспроизвести, соответствуют журналу с небольшими отклонениями.

**Эвристический алгоритм (log_fitness2):**
   - Низкий perc_fit_traces (28%) означает, что построенная эвристическим алгоритмом модель не соответствует журналу событий.
   - Средний уровень соответствия (average_trace_fitness = 0.9338458597085599) высок, что говорит о том, что трассы, которые модель смогла воспроизвести, соответствуют журналу с небольшими отклонениями.

**Индуктивный майнер (log_fitness3):**
   - Индуктивный майнер (log_fitness3) показал идеальное соответствие (perc_fit_traces = 100%, log_fitness = 1.0)

### Метрики:
- **perc_fit_traces:** Полезна для понимания, насколько модель соответствует журналу "на первый взгляд".
- **average_trace_fitness:** Помогает оценить общую "точность" модели, даже если не все трассы идеально подходят.
- **log_fitness:** Наиболее универсальная метрика, учитывающая как соответствие, так и "штрафы" за ошибки.

In [ ]:
#### Addons. Примерчик с неограниченной простенькой сеткой

from pm4py.objects.petri_net.utils import petri_utils
from pm4py.objects.petri_net.exporter import exporter as pnml_exporter
from pm4py.visualization.petri_net import visualizer as pn_visualizer

from pm4py.objects.petri_net.obj import PetriNet, Marking

# Create empty Petri net
net = PetriNet("SimpleNet")


# --- Places ---
p0 = PetriNet.Place("p0")
p1 = PetriNet.Place("p1")

net.places.update({p0, p1})

# --- Transitions ---
t1 = PetriNet.Transition("t1", "produce")
t2 = PetriNet.Transition("t2", "inflate")

net.transitions.update({t1, t2})

# --- Arcs (циклическая структура, делающая сеть неограниченной) ---
# p0 -> t1 -> p1
petri_utils.add_arc_from_to(p0, t1, net)
petri_utils.add_arc_from_to(t1, p1, net)

# p1 -> t2 -> p0 и p1 (два выхода — один цикл, второй накопление)
petri_utils.add_arc_from_to(p1, t2, net)
petri_utils.add_arc_from_to(t2, p0, net)  # возвращает маркер в p0
petri_utils.add_arc_from_to(t2, p1, net)  # генерирует дополнительный маркер → неограниченность


im = Marking()
im[p0] = 1


# Visualize
gviz = pn_visualizer.apply(net)
pn_visualizer.view(gviz)





#граф покрытия 
cg=minimal_coverability_graph.apply(net, im, final_marking)


#визуализация графа достижимости маркировок
CG = pgv.AGraph(strict=False, directed=True)
CG.graph_attr['rankdir'] = 'LR'
CG.node_attr['shape'] = 'ellipse'


for node, attributes in cg.nodes(data=True):
    marking = attributes['marking']
    for pos in marking:
        if(pos==float('inf')):
            print("Сеть неограничена!")
    CG.add_node(node, label=marking)

for e in cg.edges(data=True):
    CG.add_edge(e[0], e[1], label=e[2]['transition'], penwidth=1)
 

CG.layout(prog='dot')  # Layout algorithm (e.g., 'dot', 'neato', 'fdp', 'circo')
display(Image(CG.draw(format='png')))

In [ ]:
# Построение систем переходов 

from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.discovery.transition_system import algorithm as ts_discovery
#from examples import examples_conf
import os
import importlib.util


def execute_script():
    ts = ts_discovery.apply(simulated_log, parameters={"include_data": True})

    if importlib.util.find_spec("graphviz"):
        from pm4py.visualization.transition_system import visualizer as ts_vis
        viz = ts_vis.apply(ts, parameters={ts_vis.Variants.VIEW_BASED.value.Parameters.FORMAT: "png"})
        ts_vis.view(viz)

    for state in ts.states:
        print(state.name, "ingoing=", len(state.data["ingoing_events"]), "outgoing=",
              len(state.data["outgoing_events"]))
    for trans in ts.transitions:
        print(trans.name, trans.from_state.name, trans.to_state.name, "frequency=", len(trans.data["events"]))



execute_script()

